In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from omegaconf import OmegaConf
from atria_models.registry import MODEL
from atria_models.core.timm_model import TimmModel
from rich.pretty import pretty_repr
# model = MODEL.load_from_registry(
#     "timm",
#     overrides=["model.model_name=resnet50"],
#     provider="atria_models",
# )
model = TimmModel(model_name="resnet50")
OmegaConf.to_container(model.config)

{'_target_': 'atria_models.core.timm_model.TimmModel',
 'model_name': '???',
 'convert_bn_to_gn': True,
 'is_frozen': False,
 'frozen_keys_patterns': None,
 'unfrozen_keys_patterns': None,
 'pretrained_checkpoint': None}

In [2]:
from atria_models.registry import MODEL_PIPELINE

model_pipeline = MODEL_PIPELINE.load_from_registry(
    "image_classification",
    overrides=["model_pipeline.model.model_name=resnet50"],
    provider="atria_models",
)


/mnt/hephaistos/projects/atriav3/atria_models/.venv/lib/python3.11/site-packages/hydra/_internal/config_loader_impl.py:216: UserWarning: provider=hydra.searchpath in command-line, path=atria.conf is not available.
  warnings.warn(
/mnt/hephaistos/projects/atriav3/atria_models/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-15 22:20:33,344	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


InstantiationException: Error in call to target 'hydra_zen.funcs.zen_processing':
TypeError("TimmModel.__init__() got an unexpected keyword argument 'convert_bn_to_gn'")
full_key: model

## Creating batched instances from a list of samples
We create a list of samples and then call batched on the list which is the class method of the specific instance

In [ ]:
# Make a list of instances
instances = [
    dataset.train[i].to_tensor() for i in range(2)
]

# Batch the instances
batched = instances[0].batched(instances)

# Display the batched instances
print(batched.image.content[0] - dataset.train[0].to_tensor().image.content)
print(batched.image.content[1] - dataset.train[1].to_tensor().image.content)

[2025-07-11 12:52:15][atria_core.types.base._mixins._batchable][WARNING] Found inputs with partial None values in field ser. Ignoring field for batch.
[2025-07-11 12:52:15][atria_core.types.base._mixins._batchable][WARNING] Found inputs with partial None values in field ocr. Ignoring field for batch.
[2025-07-11 12:52:15][atria_core.types.base._mixins._batchable][WARNING] Found inputs with partial None values in field qa. Ignoring field for batch.
[2025-07-11 12:52:15][atria_core.types.base._mixins._batchable][WARNING] Found inputs with partial None values in field vqa. Ignoring field for batch.
[2025-07-11 12:52:15][atria_core.types.base._mixins._batchable][WARNING] Found inputs with partial None values in field layout. Ignoring field for batch.


tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 

## Dataset handling with File Storage
Load the dataset with a file storage manager that first caches the data into disk

In [ ]:
from atria.data.storage.file_storage_manager import FileStorageManager
from atria.data.storage.utilities import FileStorageType

# Creat a file storage manager
file_storage_manager = FileStorageManager(
    storage_dir="/tmp", streaming_mode=False, storage_type=FileStorageType.MSGPACK, 
    max_samples=100, # save up to 100 samples
)

TypeError: FileStorageManager.__init__() got an unexpected keyword argument 'storage_dir'

In [ ]:
from atria_core.types import DatasetSplitType
from atria_examples.datasets.cifar10 import Cifar10

# load the dataset with the file storage manager
cifar10 = Cifar10.load(
    split=DatasetSplitType.train,
    storage_manager=file_storage_manager,
)

AttributeError: type object 'Cifar10' has no attribute 'load'

In [ ]:
# Extract a sample instance from the dataset
cifar10[0]

ImageInstance(
    index=0,
    id=UUID('629095eb-cad3-4e9d-8b27-6e989ab27bba'),
    image=Image(
        file_path=None,
        content=<PIL.PngImagePlugin.PngImageFile image mode=RGB size=32x32 at 0x7760B0786900>,
        source_size=None,
        shape=(3, 32, 32),
        dtype=None
    ),
    label=Label(value=6, name='frog')
)